# Описание проекта

Необходимо построить модель которая будет классифицировать тексты на позитивные и негативные.

В качестве основы взята рекурентная нейронная сеть LSTM (long short-term memory - долгая краткосрочная память). Отличие данной сети от обычной RNN (recurent neiral network) в том, что она способна к обучению долговременным зависимостям.

## План проекта

1. __Анализ и подготовка данных__
    * __Загрузка и объединение данных.__ Объединим данные добавим целевой признак 'Target' со значениями 1 - негативный текст, 0 - позитивный. Для работы с данными будем пользоваться библиотекой Pandas
    * __Очистка текста от лишних знаков и сивмолов.__ Для этого подойдёт библиотека  для регулярных выражений re. Оставим только буквы.
    * __Лемматизация текста.__ При помощи библиотеки pymorphy2 будем приводить слова к их нормальной словарной форме:
        * для существительных — именительный падеж, единственное число; 
        * для прилагательных — именительный падеж, единственное число, мужской род; 
        * для глаголов, причастий, деепричастий — глагол в инфинитиве несовершенного вида.
    * __Разделение данных на тренировочную, валидационную. и тестовую__ На тренировочных данных будем учить модель, на валидационных будем проверять хорошо, ли у нас идет обучение. После обучения проверим модель на тестовых данных
    * __Векторизация текста.__ Векторизация - это перевод текста в вектор понятный компьютеру. Для этого мы воспользуемся библиотекой PyTorch.
2. __Создание модели и тренировка__
    * __Создание модели.__ Создадим следующую модель -> Embedding -> LSTM -> Dropout -> Linear -> ReLU -> Linear -> для этого будет использована библиотека PyTorch
    * __Создание функций для тренировки.__ Объединим все в одну функцию для запуска тренировки
3. __Тестирование модели__
    * __Проверка адекватности модели.__ В качестве метрики качества модели выберем F1 score
    * __Сохранение модели.__ Сохраним обученную модель для дальнейшего использования
    * __Подведение итогов.__ Оценка показателей качества модели, рекомендации по улучшению

## 1. Анализ и подготовка

In [1]:
from functions import clearing, tokenizer, DataFrameDataset, LSTM_net

In [2]:
import pandas as pd
import time

from sklearn.model_selection import train_test_split
from tqdm import tqdm

In [3]:
import torch
import torch.nn as nn
from torchtext import data

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
# Добавление воспроизводимости
SEED = 42
torch.manual_seed(SEED)
if torch.cuda.is_available(): 
    torch.cuda.manual_seed_all(SEED) 
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

### Загрузка и объединение данных

In [5]:
positive_data = pd.read_csv('data/positive.csv', sep='\n', 
                            encoding='ANSI', header=None, names=['text'])
negative_data = pd.read_csv('data/negative.csv', sep='\n', 
                            encoding='ANSI', header=None, names=['text'])

Посмотрим на данные

In [6]:
positive_data.head()

,text
0,"@first_timee хоть я и школота, но поверь, у на..."
1,"Да, все-таки он немного похож на него. Но мой ..."
2,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...
3,"""RT @digger2912: """"Кто то в углу сидит и погиб..."
4,"""@irina_dyshkant Вот что значит страшилка :D;;;;"


In [7]:
negative_data.head(5)

,text
0,"408906762813579264;""1386325944"";""dugarchikbell..."
1,"408906818262687744;""1386325957"";""nugemycejela""..."
2,"408906858515398656;""1386325966"";""4post21"";""@el..."
3,"408906914437685248;""1386325980"";""Poliwake"";""Же..."
4,"408906914723295232;""1386325980"";""capyvixowe"";""..."


In [8]:
# добавим столбец 'target' и объеденим данные
positive_data['target'] = 0
negative_data['target'] = 1
all_data = pd.concat((positive_data, negative_data), 
                     axis=0).reset_index(drop=True)

### Очистка данных
Отчистим и удалим пустые сообщения

In [9]:
# Отчистим данные
all_data['text'] = all_data['text'].apply(clearing)

In [10]:
# Удалим пропуски и сбросим индексы
all_data = all_data.dropna().reset_index(drop=True)

### Разделение данных на выборки

In [11]:
# Разделим данные на выборки, и одновременно перемешаем. 
train_val_data, test_data = train_test_split(all_data, train_size=0.8, 
                                       shuffle=True, random_state=42)

In [12]:
train_data, val_data = train_test_split(train_val_data, train_size=0.75, 
                                  shuffle=True, random_state=42)

In [13]:
train_data.shape, val_data.shape, test_data.shape

((162080, 2), (54027, 2), (54027, 2))

### Токенизация

Создадим классы с текстом и метками.

In [14]:
# Создадим поля для дальнейшей работы с библиотекой PyTorch
TEXT = data.Field(tokenize=tokenizer, include_lengths=True,)
LABEL = data.LabelField(dtype=torch.float)

In [15]:
%%time
fields = [('text',TEXT), ('label',LABEL)]
# Токенизируем наши предложжения, одновременно лемматизируем их.
train_ds, val_ds, test_ds = \
        DataFrameDataset.splits(fields, train_df=train_data, 
                                val_df=val_data, test_df=test_data)

Wall time: 9min 14s


In [16]:
# Посмотрим что получилось
print(vars(train_ds[100]))
print(vars(test_ds[100]))

{'text': ['надо', 'создать', 'акк', 'секси', 'нога', 'майкрофт', 'холмс'], 'label': 1}
{'text': ['какой', 'мерзкий', 'рожа', 'весь', 'порок', 'на', 'лицо'], 'label': 1}


### Векторизация текста

Максимальный размер словаря возьмем 75000

In [17]:
# Создадим словарь из стренировочного датасета
# Примим максимальный размер 75000 слов
MAX_VOCAB_SIZE = 75000
TEXT.build_vocab(train_ds, max_size = MAX_VOCAB_SIZE)
LABEL.build_vocab(train_ds)

In [18]:
print(f'Итоговый размер словаря: {len(TEXT.vocab)}')

Итоговый размер словаря: 73567


### Вывод:
1. Данные были загружены и объеденены. Добавлен столбец с классами 'target', в котором 0 - это позитивный текст, 1 - негативный.
4. Данные разделены на тренировочную - 60%, валидационную - 20% и тестовую - 20% выборки.
2. Проведена лемматизация и отчистка данных. В результате остались только русские символы. Слова приведены к нормальной словоформе.
3. Текст был векторизирован. Размер словаря 73567 слов

## 2. Создание модели и тренировка

### Создание функций для тренировки

In [19]:
BATCH_SIZE = 128
# Создание итератора 
# Итератор разбивает данные на батчи для последовательной подачи их в модель
train_iterator, valid_iterator, test_iterator = \
        data.BucketIterator.splits(
    (train_ds, val_ds, test_ds), 
    batch_size = BATCH_SIZE,
    sort_within_batch = True,
    device = device)

Напишем функцию F1 меры: $F1 = \frac{2\cdot  precision \cdot recall}{precision + recall}$

In [20]:
def f1_score(preds, ground_truth):
    """
    Функция высчитыавет F1 score
    :param preds: Предсказания модели
    :param ground_truth: Метки истинности
    :return: значение f1
    """
    # Высчитываем вероятности и округляем их
    predictions = torch.round(torch.sigmoid(preds))
    # Считаем TP FP и FN
    true_positive = ((predictions == 1) * (ground_truth == 1)).sum().item()
    false_positive = ((predictions == 1) * (ground_truth == 0)).sum().item()
    false_negative = ((predictions == 0) * (ground_truth == 1)).sum().item()
    # Исключаем случаи деления на ноль
    if true_positive != 0 and false_positive != 0:
        precision = true_positive / (true_positive + false_positive)
    else:
        precision = 1
        
    if true_positive != 0 and false_negative != 0:
        recall = true_positive / (true_positive + false_negative)
    else:
        recall = 1

    return 2 * precision * recall / (precision + recall)

In [21]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

Напишем функцию тренировки модели.

In [22]:
def train(model, iterator, name):
    """
    Функция для тренировки модели
    :param model: Модель
    :param iterator: Итератор данных
    :param name: Префикс для вывода информации о тренировки
    :return: Средний loss и средний score
    """

    epoch_loss = 0
    epoch_acc = 0

    # Переведем модель в режим обучения
    model.train()
    # Создадим панель статуса
    with tqdm(total=len(iterator)) as nb:
        # Цикл по итератору
        for ind, batch in enumerate(iterator):
            # Получаем векторизированный текст и его длину
            text, text_lengths = batch.text
            # Обнулим градиенты у весов модели
            optimizer.zero_grad()
            # Получим сырые предсказания
            predictions = model(text, text_lengths).squeeze(1)
            # Высчитаем loss
            loss = criterion(predictions, batch.label)
            # Посчитаем F1 score
            acc = binary_accuracy(predictions, batch.label)

            # Посчитаем ошибки для весов
            loss.backward()
            # Обновим веса
            optimizer.step()

            epoch_loss += loss.item()
            epoch_acc += acc
            description ='T: ' + name + f', loss - {epoch_loss / (ind + 1):.4f}' + f', F1 - {epoch_acc / (ind + 1):.2%}'
            # обновим панель статуса
            nb.set_description(desc=description, refresh=True)
            nb.update()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

Функция для предсказания модели

In [23]:
def evaluate(model, iterator):
    """
    Функция для предсказаний без обученя
    :param model: Модель
    :param iterator: Итератор
    :return: Среднее значение F1 score
    """
    epoch_acc = 0
    # Перевод модели в режим предсказания
    model.eval()
    # Создадим панель статуса
    with tqdm(total=len(iterator)) as nb:
        # Выключим градиенты
        with torch.no_grad():
            # Цикл по итератору
            for ind, batch in enumerate(iterator):
                text, text_lengths = batch.text
                predictions = model(text, text_lengths).squeeze(1)
                acc = binary_accuracy(predictions, batch.label)

                epoch_acc += acc
                nb.set_description(desc=f'V: F1 - {epoch_acc / (ind + 1):.2%}', 
                                   refresh=True)
                nb.update()
    return epoch_acc / len(iterator)

### Тренировка

Опишем саму модель: -> Embedding -> LSTM -> Dropout -> Linear -> ReLU -> Linear ->:
    * Embedding - преобразует вектор словаря в вещественный вектор в пространстве с фиксированной невысокой размерностью.
    * LSTM - Рекуррентная сеть
    * Dropout - Слой позволяющий случайно занулять выходы из предыдущего слоя для регуляризации и лучшего обучения
    * Linear - Обычный линейный слой
    * ReLU - Нелинейная функция

In [51]:
# Настройки тренировки
num_epochs = 3 # Количество эпох для тренировки
learning_rate = 0.001 # Коэффициент обучения

INPUT_DIM = len(TEXT.vocab) # Входной вектор
EMBEDDING_DIM = 200 # Выходной вектор из Embedding
HIDDEN_DIM = 256 # Размерность слоев LSTM
OUTPUT_DIM = 1 # Выходной вектор
N_LAYERS = 2 # Колличество слоев в LSTM
BIDIRECTIONAL = True # Двунаправленная LSTM
DROPOUT = 0.2 # Процент случайного зануления
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token] # Паддинг

In [60]:
# Создадим модель
model = LSTM_net(INPUT_DIM, 
            EMBEDDING_DIM, 
            HIDDEN_DIM, 
            OUTPUT_DIM, 
            N_LAYERS, 
            BIDIRECTIONAL, 
            DROPOUT, 
            PAD_IDX).to(device)

model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

В качестве функции ошибки возьмем BCEWithLogLoss

In [61]:
param_optimizer = list(model.named_parameters())
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if n == 'embedding.weight'], 
     'learning_rate': 1e-1
    },
    {'params': [p for n, p in param_optimizer if n != 'embedding.weight'],
     'learning_rate': 1e-3,
     'weight_decay': 0.002
    }
]

In [62]:
# Создадим функцию ошибки и оптимизатор
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(optimizer_grouped_parameters)

In [ ]:
# Процесс тренировки
t = time.time()
# Будем сохранять историю
loss=[]
acc=[]
val_acc=[]

# Цикл по эпохам
for epoch in range(num_epochs):
    name = f'Epoch [{epoch + 1} / {num_epochs}]'
    # Тренировка
    train_loss, train_acc = train(model, train_iterator, name)
    # Проверка на валидации
    valid_acc = evaluate(model, valid_iterator)
    # Запись истории
    loss.append(train_loss)
    acc.append(train_acc)
    val_acc.append(valid_acc)
    
print(f'time:{time.time()-t:.3f}')

T: Epoch [1 / 3], loss - 0.6144, F1 - 66.09%: 100%|█████████▉| 1266/1267 [01:08<00:00, 19.82it/s]

### Вывод:
1. Для проверки точности модели была взята функция F1 мера.
2. В качестве Loss функции взята BCEWithLogLoss
3. После обучения модели можно заметить, что значение F1 на тренировочной выборке равняется 79.16%, а на валидационной выборке 71.09%. Модель начала запоминать тренировочные данные. Для решения данной проблемы воможно уменьшить размерность модели. Добавить регуляризации. Взять уже натренированный Embedding.

## 3. Тестирование модели

### Проверка модели 

In [ ]:
# Посмотрим как модель предсказывает на тестовой выборке
test_acc = evaluate(model, test_iterator)

### Сохранение словаря

In [ ]:
def save_vocab(vocab, path):
    import pickle
    output = open(path, 'wb')
    pickle.dump(vocab, output)
    output.close()

In [ ]:
# Сохраним значения словаря в файл
save_vocab(TEXT.vocab, 'vocab.pkl')

### Сохранение модели

Сохраним веса модели в документ. В дальнейшем можно  будет загрузить уже натренированные веса 

In [ ]:
# Сохраним веса модели и параметры с которыми эта модель была обучена
torch.save({'model_state_dict': model.state_dict(),
            'input_dim': INPUT_DIM, 
            'embedding_dim': EMBEDDING_DIM, 
            'hidden_dim': HIDDEN_DIM, 
            'output_dim': OUTPUT_DIM, 
            'n_layers': N_LAYERS, 
            'bidirectional': BIDIRECTIONAL, 
            'dropout': DROPOUT, 
            'pad_idx': PAD_IDX}, 'model.torch')

## Вывод:
1. Предсказания на тестовой выборке показали F1 меру 71.57%, что соответствует точности на валидационной выборке.
2. Был сохранен словарь и веса модели, для дальнейшего их использования с другими данными

## Выводы:

1. В ходе работы были подготовлены данные состоящие из позитивных и негативных комментариев. Для начала из данных сформирован один датасет к которому добален признак "target" со значениями 0 или 1, где 0 - прзитивный комментарий, 1 - негативный. 
1. Текст очищен от лишних знаков, слова приведены к нормальной словарной форме. После был составлен словарь, размер которого получился 73567 слов. Данные разделены на тренировочную, валидационную, и тестовую выборки. 
1. Текст был векторизирован для подачи в подель
2. Метрикой качества была выбрана функция F1 мера.
2. После трех эпох тренировок модель показала значение F1 на тренировочной выборке равным 79.16%, а на валидационной 71.09%. Что является не большим значением. Для достижения лучшего значения следует рассмотреть рассомтреть следующие варианты:
    * Добавить регуляризации модели
    * Изменить метод инициализации весов модели
    * Сделать разные коэффициенты обучения для различных слоев модели. Например для слоя Embedding увеличить коэффициент обучения.
    * Добавление Attention слоя
3. На тестовой выборке модель показала знаение F1 - 71.57%.
3. Словарь сохранен в файл vocab.txt, а веса модели сохранены в файл model.torch